# 1. Import Dependencies
In this section, we are importing the necessary Python libraries that our program will depend on. Here is what each one does:

- cv2 (OpenCV): This is an open-source computer vision and machine learning software library. OpenCV is used to manipulate images and videos, including capturing images and video footage in real time using device cameras. It's one of the most popular tools for this kind of work due to its wide array of capabilities and its performance. Here, we install it with !pip install opencv-python and import it as cv2.

- uuid (Universally Unique Identifier): This is a Python library that allows us to generate unique ids. We'll use this to generate unique names for our image files.

- os (Operating System): This built-in Python library enables us to interface with the underlying operating system that Python is running on (Windows, Mac, Linux). We can use it to manage files, navigate the file system, and get system information, among other things. We will use it mainly for file and directory operations.

- time: This is another built-in Python library that provides functionality for working with times, and for performing time-related operations, like pausing the execution of the program for a certain amount of time.

In [ ]:
!pip install opencv-python

In [ ]:
# Import opencv
import cv2 

# Import uuid
import uuid

# Import Operating System
import os

# Import time
import time

In this code block, we start by installing the OpenCV library using pip. Pip (Pip Installs Packages) is a package management system used to install and manage software packages written in Python.

We then import the necessary modules that we need for our program:

- cv2: This is our OpenCV library which we will use for capturing and manipulating our images.

- uuid: This module will be used to generate unique identifiers for our images.

- os: We'll use this module to interact with the operating system, for tasks like creating new directories to store our images.

- time: This module will be used to control the timing of certain operations, for example, pausing between capturing images to give us time to reposition the objects being photographed.

# 2. Define Images to Collect
In this part of the script, we define the labels for the types of images we want to collect. We also set the number of images we plan to collect for each label. Here's what the code looks like:


In [ ]:
labels = ['screw', 'nut']
number_imgs = 40

In this code block, we're creating a list called labels. Each item in the list is a label for a type of image that we want to collect. In this case, we're collecting images of screws and nuts.

The number_imgs variable is the number of images we want to collect for each label. Here, we're collecting 40 images each of screws and nuts.

This means that the program will attempt to capture and save 40 images each for 'screw' and 'nut', giving us a total of 80 images. This dataset can then be used for tasks such as machine learning model training.

You can change these variables to suit your needs. For example, if you want to capture images of different objects, you can modify the labels list. If you want to capture more or fewer images, you can modify the number_imgs variable.

# 3. Setup Folders
The next step is to set up the directories where we'll store the collected images. We define a specific path for this purpose, and if the path doesn't exist yet, we create it.


In [ ]:
IMAGES_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'collectedimages')

In [ ]:
if not os.path.exists(IMAGES_PATH):
    if os.name == 'posix':
        !mkdir -p {IMAGES_PATH}
    if os.name == 'nt':
         !mkdir {IMAGES_PATH}
for label in labels:
    path = os.path.join(IMAGES_PATH, label)
    if not os.path.exists(path):
        !mkdir {path}

Here, IMAGES_PATH is a string that specifies where we want to save our images. We use the os.path.join() function to create this path. This function is useful because it can create paths that work on any operating system.

Next, we check if the path specified by IMAGES_PATH exists. If it doesn't exist, we create it. The way we create the directory depends on the operating system.

The os.name variable is a string that indicates which operating system you're using. If os.name == 'posix', you're on a Unix-like operating system like Linux or Mac OS X. If os.name == 'nt', you're on Windows.

After creating the main directory for the images, we also create subdirectories for each of the labels we defined earlier (screw and nut). This way, we have a separate directory for each type of image, making our collected data more organized.

# 4. Capture Images
In this step, we start capturing images for each label using the device's primary camera.

In [ ]:
for label in labels:
    cap = cv2.VideoCapture(0)
    print('Collecting images for {}'.format(label))
    time.sleep(5)
    for imgnum in range(number_imgs):
        print('Collecting image {}'.format(imgnum))
        ret, frame = cap.read()
        imgname = os.path.join(IMAGES_PATH,label,label+'.'+'{}.jpg'.format(str(uuid.uuid1())))
        cv2.imwrite(imgname, frame)
        cv2.imshow('frame', frame)
        time.sleep(2)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

We first initialize the camera with cv2.VideoCapture(0), then for each label in the list of labels, we start collecting the specified number of images (number_imgs).

For each image, we read a frame from the camera, construct a unique image name using the label and a unique ID (using the uuid library), and save it in the appropriate folder with cv2.imwrite(). We also display the frame for 2 seconds using cv2.imshow().

If at any point the 'q' key is pressed, the image collection for the current label stops.

Once all images have been collected, we release the camera with cap.release() and close any OpenCV windows with cv2.destroyAllWindows().

# 5. Image Labelling
In this step, we label our collected images using the LabelImg tool, an open source graphical image annotation tool written in Python and Qt. It's a great tool for labelling objects in images for use in machine learning applications.

First, we install PyQt5 and lxml, which are required for running LabelImg.

Then, we set up the path to LabelImg and clone the tool from its GitHub repository if it doesn't already exist in that location.

Finally, we start the LabelImg tool.

In [ ]:
!pip install --upgrade pyqt5 lxml

In [ ]:
LABELIMG_PATH = os.path.join('Tensorflow', 'labelimg')

In [ ]:
if not os.path.exists(LABELIMG_PATH):
    !mkdir {LABELIMG_PATH}
    !git clone https://github.com/tzutalin/labelImg {LABELIMG_PATH}

In [ ]:
if os.name == 'posix':
    !cd {LABELIMG_PATH} && make qt5py3
if os.name =='nt':
    !cd {LABELIMG_PATH} && pyrcc5 -o libs/resources.py resources.qrc

In [ ]:
!cd {LABELIMG_PATH} && python labelImg.py

If you're running a POSIX-compliant operating system (like Linux or MacOS), we use the make qt5py3 command to build LabelImg. For Windows systems, we use the pyrcc5 -o libs/resources.py resources.qrc command. After this, we start LabelImg with the command python labelImg.py.

LabelImg allows you to draw bounding boxes on your images and label them. You can then save these labels in XML format, which can be used to train your object detection model.

# 6. Move them into a Training and Testing Partition
After collecting and labeling our images, we need to organize them into separate training and testing directories. In machine learning, it's common to split the available data into a training set and a testing set. The training set is used to train the model, and the testing set is used to evaluate the model's performance.

We define the paths to the training and testing directories and also specify the path to the archive file we will create.

# OPTIONAL - 7. Compress them for Colab Training

In [ ]:
TRAIN_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'train')
TEST_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'test')
ARCHIVE_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'archive.tar.gz')

In the next steps (which will be covered in the following Jupyter notebooks), we will move our labeled images into these directories and create a compressed archive of them. This is especially useful if you plan to train your model on a cloud-based platform like Google Colab, as it allows for easy uploading of your entire dataset.

Keep in mind that you need to manually distribute your images into the 'train' and 'test' folders with a typical split of 70-30 or 80-20. This split ensures that your model gets a diverse set of images to learn from and a separate, equally diverse set of images to test its learning.

In [ ]:
!tar -czf {ARCHIVE_PATH} {TRAIN_PATH} {TEST_PATH}